In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz

In [8]:

    

!tar xf spark-3.3.1-bin-hadoop3.tgz


    

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"



!pip install -q findspark
!pip install pyspark -q
import findspark
findspark.init()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.4 MB/s eta 0:00:00


In [4]:
findspark.find()


'/content/spark-3.3.1-bin-hadoop3'

In [13]:
#zwykłe sumowanie 
from pyspark.sql import SparkSession
from pyspark.sql import Row

# tworzymy SparkSession zamiast context
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()
# getOrCreate tworzy nową sesję albo podłącza się do istniejącej

def mapper(line):
    fields = line.split(',')
    return Row(id=int(fields[0]), 
               item_id=int(fields[1]), amount=float(fields[2]))

# dane nie mają nagłówka, dlatego korzystamy z contextu żeby załadować dane
lines = spark.sparkContext.textFile("customer-orders.csv")
# nasza funkcja przekonwertuje nasze dane na wiersze i pola z nazwami kolumn i typami danych
orders = lines.map(mapper)

# tworzymy dataframe
# cacheujemy ponieważchcemy wysyłać wiele zapytań i trzeba to jakoś przechować w pamięci
schemaOrders = spark.createDataFrame(orders).cache()
# tworzymy widok jak w bazie danych aby wysyłać zapytania, widok people działa jak tabela
schemaOrders.createOrReplaceTempView("orders")

# teraz możemy wysyłać zapytania do DF jak do zwykłej tabeli w SQL
# nazwy kolumn to nazwy pól. które nazwaliśmy
total = spark.sql("SELECT id, SUM(amount) FROM orders GROUP BY id")

# wynik zapytań SQL można potem wrzucić do podstawowych funkcji RDD, tu zbieramy wyniki i je wyświetlamy
for amount in total.collect():
  print(amount)

# zamiast pełnego zapytania możemy również korzystać z funkcji
#schemaPeople.groupBy("age").count().orderBy("age").show()

#spark.stop()


Row(id=29, sum(amount)=5032.530000000001)
Row(id=26, sum(amount)=5250.4)
Row(id=65, sum(amount)=5140.35)
Row(id=54, sum(amount)=6065.390000000001)
Row(id=19, sum(amount)=5059.43)
Row(id=0, sum(amount)=5524.949999999999)
Row(id=22, sum(amount)=5019.449999999999)
Row(id=7, sum(amount)=4755.069999999999)
Row(id=77, sum(amount)=4327.73)
Row(id=34, sum(amount)=5330.8)
Row(id=50, sum(amount)=4517.2699999999995)
Row(id=94, sum(amount)=4475.570000000001)
Row(id=57, sum(amount)=4628.4)
Row(id=43, sum(amount)=5368.83)
Row(id=32, sum(amount)=5496.05)
Row(id=84, sum(amount)=4652.9400000000005)
Row(id=31, sum(amount)=4765.049999999999)
Row(id=98, sum(amount)=4297.259999999999)
Row(id=39, sum(amount)=6193.110000000001)
Row(id=25, sum(amount)=5057.610000000001)
Row(id=95, sum(amount)=4876.839999999998)
Row(id=71, sum(amount)=5995.660000000002)
Row(id=6, sum(amount)=5397.879999999999)
Row(id=68, sum(amount)=6375.450000000001)
Row(id=72, sum(amount)=5337.439999999999)
Row(id=87, sum(amount)=5206.4)
Row

In [15]:
#posortowane
from pyspark.sql import SparkSession
from pyspark.sql import Row

# tworzymy SparkSession zamiast context
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()
# getOrCreate tworzy nową sesję albo podłącza się do istniejącej

def mapper(line):
    fields = line.split(',')
    return Row(id=int(fields[0]), 
               item_id=int(fields[1]), amount=float(fields[2]))

# dane nie mają nagłówka, dlatego korzystamy z contextu żeby załadować dane
lines = spark.sparkContext.textFile("customer-orders.csv")
# nasza funkcja przekonwertuje nasze dane na wiersze i pola z nazwami kolumn i typami danych
orders = lines.map(mapper)

# tworzymy dataframe
# cacheujemy ponieważchcemy wysyłać wiele zapytań i trzeba to jakoś przechować w pamięci
schemaOrders = spark.createDataFrame(orders).cache()
# tworzymy widok jak w bazie danych aby wysyłać zapytania, widok people działa jak tabela
schemaOrders.createOrReplaceTempView("orders")

# teraz możemy wysyłać zapytania do DF jak do zwykłej tabeli w SQL
# nazwy kolumn to nazwy pól. które nazwaliśmy
total = spark.sql("SELECT id, SUM(amount) as total FROM orders GROUP BY id ORDER BY total")

# wynik zapytań SQL można potem wrzucić do podstawowych funkcji RDD, tu zbieramy wyniki i je wyświetlamy
for amount in total.collect():
  print(amount)

# zamiast pełnego zapytania możemy również korzystać z funkcji
#schemaPeople.groupBy("age").count().orderBy("age").show()

#spark.stop()

Row(id=45, total=3309.3799999999997)
Row(id=79, total=3790.5699999999997)
Row(id=96, total=3924.2300000000005)
Row(id=23, total=4042.65)
Row(id=99, total=4172.29)
Row(id=75, total=4178.5)
Row(id=36, total=4278.049999999999)
Row(id=98, total=4297.259999999999)
Row(id=47, total=4316.299999999998)
Row(id=77, total=4327.73)
Row(id=13, total=4367.619999999999)
Row(id=48, total=4384.33)
Row(id=49, total=4394.6)
Row(id=94, total=4475.570000000001)
Row(id=67, total=4505.79)
Row(id=50, total=4517.2699999999995)
Row(id=78, total=4524.51)
Row(id=5, total=4561.07)
Row(id=57, total=4628.4)
Row(id=83, total=4635.8)
Row(id=91, total=4642.26)
Row(id=74, total=4647.130000000001)
Row(id=84, total=4652.9400000000005)
Row(id=3, total=4659.63)
Row(id=12, total=4664.589999999999)
Row(id=66, total=4681.92)
Row(id=56, total=4701.02)
Row(id=21, total=4707.41)
Row(id=80, total=4727.86)
Row(id=14, total=4735.030000000001)
Row(id=37, total=4735.200000000001)
Row(id=7, total=4755.069999999999)
Row(id=44, total=475